In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df_3 = pd.read_csv('s3://datasetairbnb/dataset/listingsMarch.csv', encoding='Latin-1', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
df_4 = pd.read_csv('s3://datasetairbnb/dataset/listingsApril.csv', encoding='Latin-1', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
df_5 = pd.read_csv('s3://datasetairbnb/dataset/listingsMay.csv', encoding='Latin-1', sep=',', error_bad_lines=False, index_col=False, low_memory=False)

df_15 = pd.read_csv('s3://datasetairbnb/dataset/listings_2017/march.csv', encoding='Latin-1', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
df_16 = pd.read_csv('s3://datasetairbnb/dataset/listings_2017/april.csv', encoding='Latin-1', sep=',', error_bad_lines=False, index_col=False, low_memory=False)
df_17 = pd.read_csv('s3://datasetairbnb/dataset/listings_2017/may.csv', encoding='Latin-1', sep=',', error_bad_lines=False, index_col=False, low_memory=False)


In [3]:
df_spring_seasonality = pd.concat([df_3, df_4, df_5, df_15, df_16, df_17], ignore_index=True)

In [4]:
df_spring = df_spring_seasonality[[
    'id',
    'host_id',
    'scrape_id',
    'zipcode',
    'neighbourhood_cleansed',
    'property_type',
    'room_type',
    'accommodates',
    'bedrooms',
    'beds',
    'bed_type',
    'price',
    'number_of_reviews',
    'review_scores_rating',
    'availability_30',
    'minimum_nights',
    'bathrooms',
    'host_is_superhost',
    'host_response_rate'
]]


In [5]:
df_spring.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 266811 entries, 0 to 266810
Data columns (total 19 columns):
id                        266811 non-null int64
host_id                   266811 non-null int64
scrape_id                 266811 non-null int64
zipcode                   262937 non-null object
neighbourhood_cleansed    266811 non-null object
property_type             266811 non-null object
room_type                 266811 non-null object
accommodates              266811 non-null int64
bedrooms                  266435 non-null float64
beds                      266378 non-null float64
bed_type                  266811 non-null object
price                     266811 non-null object
number_of_reviews         266811 non-null int64
review_scores_rating      201920 non-null float64
availability_30           266811 non-null int64
minimum_nights            266811 non-null int64
bathrooms                 265952 non-null float64
host_is_superhost         266215 non-null object
host_respo

In [6]:
# determine the number of missing entries in each column
for col in df_spring.columns:
    print (col + ', Number of Missing Values:', len(df_spring[col][df_spring[col].isnull()]))

id, Number of Missing Values: 0
host_id, Number of Missing Values: 0
scrape_id, Number of Missing Values: 0
zipcode, Number of Missing Values: 3874
neighbourhood_cleansed, Number of Missing Values: 0
property_type, Number of Missing Values: 0
room_type, Number of Missing Values: 0
accommodates, Number of Missing Values: 0
bedrooms, Number of Missing Values: 376
beds, Number of Missing Values: 433
bed_type, Number of Missing Values: 0
price, Number of Missing Values: 0
number_of_reviews, Number of Missing Values: 0
review_scores_rating, Number of Missing Values: 64891
availability_30, Number of Missing Values: 0
minimum_nights, Number of Missing Values: 0
bathrooms, Number of Missing Values: 859
host_is_superhost, Number of Missing Values: 596
host_response_rate, Number of Missing Values: 92912


In [7]:
original_winter = len(df_spring)
df_spring = df_spring.dropna(how='any', subset=['zipcode', 'property_type', 'bedrooms', 'beds', 'bathrooms'])
print ('Number of NaN values removed:', original_winter - len(df_spring))


Number of NaN values removed: 5174


In [8]:
# drop $ and , 
df_spring['price'] = df_spring['price'].astype(str).str.replace('$', '')
df_spring['price'] = df_spring['price'].astype(str).str.replace(',', '')
df_spring['price'] = df_spring['price'].str.strip()
df_spring['price'] = df_spring['price'].astype(float)


In [9]:
cleanup_proptype = {'Apartment': 1, 'House': 2, 'Townhouse': 3, 'Loft': 4, 'Condominium': 5, 'Serviced apartment': 6,  'Guest suite': 7, 'Other': 8,
                     'Bed and breakfast': 9, 'Bed & Breakfast': 9, 'Boutique hotel': 10, 'Resort': 11, 'Guesthouse': 12, 'Hostel': 13, 'Hotel': 14, 'Bungalow': 15,
                    'Villa': 16, 'Tiny house': 17, 'Aparthotel': 18, 'Boat': 19, 'Cottage': 20, 'Tent': 21, 'Camper/RV': 22, 'Cabin': 23, 'Casa particular (Cuba)': 24,
                   'Castle': 25, 'Houseboat': 26, 'Earth house': 27, 'Nature lodge': 28, 'Cave': 29, 'Train': 30, 'Timeshare': 31, 'Island': 32, 'Bus': 33, 'Chalet': 34
                   }
df_spring['property_type'].replace(cleanup_proptype, inplace=True)

In [10]:
# drop % && convert to float
df_spring['host_response_rate'].fillna(0, inplace=True)
df_spring['host_response_rate'] = df_spring['host_response_rate'].astype(str).str.replace('%', '')
df_spring['host_response_rate'] = df_spring['host_response_rate'].astype(str).str.replace(',', '')
df_spring['host_response_rate'] = df_spring['host_response_rate'].str.strip()
df_spring['host_response_rate'] = df_spring['host_response_rate'].astype(float)

In [11]:
# host_is_superhost
df_spring.host_is_superhost[df_spring.host_is_superhost == 't'] = 1
df_spring.host_is_superhost[df_spring.host_is_superhost == 'f'] = 0
df_spring.host_is_superhost = df_spring.host_is_superhost.astype(float)


threshold=100
df['count'] = df_cols.groupby('property_type')['property_type'].transform(pd.Series.value_counts)
df_cols=df[df['count']>=threshold]
df_cols.drop(['count'], axis = 1, inplace = True)

In [12]:
df_spring = df_spring.groupby('property_type').filter(lambda x : len(x)>500)

In [13]:
df_spring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259382 entries, 0 to 266809
Data columns (total 19 columns):
id                        259382 non-null int64
host_id                   259382 non-null int64
scrape_id                 259382 non-null int64
zipcode                   259382 non-null object
neighbourhood_cleansed    259382 non-null object
property_type             259382 non-null object
room_type                 259382 non-null object
accommodates              259382 non-null int64
bedrooms                  259382 non-null float64
beds                      259382 non-null float64
bed_type                  259382 non-null object
price                     259382 non-null float64
number_of_reviews         259382 non-null int64
review_scores_rating      196662 non-null float64
availability_30           259382 non-null int64
minimum_nights            259382 non-null int64
bathrooms                 259382 non-null float64
host_is_superhost         258811 non-null float64
host_res

In [14]:
df_spring['neighbourhood_cleansed']

0                     Harlem
1                 Kensington
2                    Midtown
3               Williamsburg
4                     Harlem
5               Clinton Hill
6             Hell's Kitchen
7             Hell's Kitchen
9                Sunset Park
10               Murray Hill
11            Hell's Kitchen
12           Upper West Side
13                 Chinatown
14           Upper West Side
15           Upper West Side
16            Hell's Kitchen
18           Upper West Side
19              West Village
21               East Harlem
22               Fort Greene
24                   Chelsea
26           Upper West Side
27                Park Slope
28                Park Slope
29                Park Slope
30                    Harlem
31              East Village
32        Bedford-Stuyvesant
33              East Village
35                    Inwood
                 ...        
266779          Williamsburg
266780          Williamsburg
266781          Williamsburg
266782        

In [15]:
df_spring['neighbourhood_cleansed'] = df_spring['neighbourhood_cleansed'].str.lower()
df_spring.neighbourhood_cleansed = df_spring.neighbourhood_cleansed.str.replace(' ', '')

In [16]:
df_spring['neighbourhood_cleansed']

0                     harlem
1                 kensington
2                    midtown
3               williamsburg
4                     harlem
5                clintonhill
6              hell'skitchen
7              hell'skitchen
9                 sunsetpark
10                murrayhill
11             hell'skitchen
12             upperwestside
13                 chinatown
14             upperwestside
15             upperwestside
16             hell'skitchen
18             upperwestside
19               westvillage
21                eastharlem
22                fortgreene
24                   chelsea
26             upperwestside
27                 parkslope
28                 parkslope
29                 parkslope
30                    harlem
31               eastvillage
32        bedford-stuyvesant
33               eastvillage
35                    inwood
                 ...        
266779          williamsburg
266780          williamsburg
266781          williamsburg
266782        

In [17]:
df_spring.room_type[df_spring.room_type == 'Entire home/apt'] = 1.0000000
df_spring.room_type[df_spring.room_type == 'Private room'] = 1.01
df_spring.room_type[df_spring.room_type == 'Shared room'] = 1.02
df_spring.room_type = df_spring.room_type.astype(float)


In [18]:
cleanup_proptype = {'Apartment': 1, 'House': 2, 'Townhouse': 3, 'Loft': 4, 'Condominium': 5, 'Serviced apartment': 6,  'Guest suite': 7, 'Other': 8,
                     'Bed and breakfast': 9, 'Bed & Breakfast': 9, 'Boutique hotel': 10, 'Resort': 11, 'Guesthouse': 12, 'Hostel': 13, 'Hotel': 14, 'Bungalow': 15,
                    'Villa': 16, 'Tiny house': 17, 'Aparthotel': 18, 'Boat': 19, 'Cottage': 20, 'Tent': 21, 'Camper/RV': 22, 'Cabin': 23, 'Casa particular (Cuba)': 24,
                   'Castle': 25, 'Houseboat': 26, 'Earth house': 27, 'Nature lodge': 28, 'Cave': 29, 'Train': 30, 'Timeshare': 31, 'Island': 32, 'Bus': 33, 'Chalet': 34
                   }
df_spring['property_type'].replace(cleanup_proptype, inplace=True)

In [19]:
df_spring.bed_type[df_spring.bed_type == 'Real Bed'] = 1.0000000
df_spring.bed_type[df_spring.bed_type == 'Futon'] = 1.01
df_spring.bed_type[df_spring.bed_type == 'Pull-out Sofa'] = 1.02
df_spring.bed_type[df_spring.bed_type == 'Airbed'] = 1.03
df_spring.bed_type[df_spring.bed_type == 'Couch'] = 1.04
df_spring.bed_type = df_spring.bed_type.astype(float)
df_spring.bed_type

0         1.00
1         1.00
2         1.00
3         1.00
4         1.02
5         1.00
6         1.00
7         1.00
9         1.00
10        1.00
11        1.00
12        1.00
13        1.00
14        1.00
15        1.00
16        1.00
18        1.03
19        1.00
21        1.00
22        1.00
24        1.00
26        1.00
27        1.00
28        1.00
29        1.00
30        1.00
31        1.00
32        1.00
33        1.02
35        1.00
          ... 
266779    1.00
266780    1.00
266781    1.00
266782    1.00
266783    1.00
266784    1.00
266785    1.00
266786    1.00
266787    1.00
266788    1.00
266789    1.00
266790    1.00
266791    1.00
266792    1.00
266793    1.00
266794    1.00
266795    1.00
266796    1.00
266797    1.00
266798    1.00
266799    1.00
266800    1.00
266801    1.00
266802    1.00
266803    1.00
266804    1.00
266805    1.00
266806    1.00
266807    1.00
266809    1.00
Name: bed_type, Length: 259382, dtype: float64

In [20]:
df_spring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259382 entries, 0 to 266809
Data columns (total 19 columns):
id                        259382 non-null int64
host_id                   259382 non-null int64
scrape_id                 259382 non-null int64
zipcode                   259382 non-null object
neighbourhood_cleansed    259382 non-null object
property_type             259382 non-null int64
room_type                 259382 non-null float64
accommodates              259382 non-null int64
bedrooms                  259382 non-null float64
beds                      259382 non-null float64
bed_type                  259382 non-null float64
price                     259382 non-null float64
number_of_reviews         259382 non-null int64
review_scores_rating      196662 non-null float64
availability_30           259382 non-null int64
minimum_nights            259382 non-null int64
bathrooms                 259382 non-null float64
host_is_superhost         258811 non-null float64
host_re

In [21]:
df_spring['host_is_superhost'] = df_spring['host_is_superhost'].fillna(0)

In [22]:
df_spring.review_scores_rating.mean()

93.35948988620069

In [23]:
df_spring.review_scores_rating = df_spring.review_scores_rating.fillna(df_spring.review_scores_rating.mean())

In [24]:
df_spring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259382 entries, 0 to 266809
Data columns (total 19 columns):
id                        259382 non-null int64
host_id                   259382 non-null int64
scrape_id                 259382 non-null int64
zipcode                   259382 non-null object
neighbourhood_cleansed    259382 non-null object
property_type             259382 non-null int64
room_type                 259382 non-null float64
accommodates              259382 non-null int64
bedrooms                  259382 non-null float64
beds                      259382 non-null float64
bed_type                  259382 non-null float64
price                     259382 non-null float64
number_of_reviews         259382 non-null int64
review_scores_rating      259382 non-null float64
availability_30           259382 non-null int64
minimum_nights            259382 non-null int64
bathrooms                 259382 non-null float64
host_is_superhost         259382 non-null float64
host_re

In [25]:
df_spring.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259382 entries, 0 to 266809
Data columns (total 19 columns):
id                        259382 non-null int64
host_id                   259382 non-null int64
scrape_id                 259382 non-null int64
zipcode                   259382 non-null object
neighbourhood_cleansed    259382 non-null object
property_type             259382 non-null int64
room_type                 259382 non-null float64
accommodates              259382 non-null int64
bedrooms                  259382 non-null float64
beds                      259382 non-null float64
bed_type                  259382 non-null float64
price                     259382 non-null float64
number_of_reviews         259382 non-null int64
review_scores_rating      259382 non-null float64
availability_30           259382 non-null int64
minimum_nights            259382 non-null int64
bathrooms                 259382 non-null float64
host_is_superhost         259382 non-null float64
host_re

In [26]:
from io import StringIO
import boto3

csv_buffer = StringIO()
df_spring.to_csv(csv_buffer)
s3_resource = boto3.resource('s3')
s3_resource.Object("datasetairbnb", 'airbnb_cleaned_spring.csv').put(Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': 'BD7826540D9F7B15',
  'HostId': 'ONnq//3LukwDYitCMBvTV1Pw9pkbY195H2HzwURVElSYwqymHYNaQ1FiKRWw4OETFEWJd3RD60U=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'ONnq//3LukwDYitCMBvTV1Pw9pkbY195H2HzwURVElSYwqymHYNaQ1FiKRWw4OETFEWJd3RD60U=',
   'x-amz-request-id': 'BD7826540D9F7B15',
   'date': 'Wed, 15 May 2019 05:03:43 GMT',
   'etag': '"5ed4168fcd7b34d9486a764db9f18f49"',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"5ed4168fcd7b34d9486a764db9f18f49"'}